# Data / Synchronisation

## Synchronisation des données

Les indices pouvant être utilisés pour vérifier la synchronisation des données sont:
- des durées entre les deux enregistrements cohérentes.
- des pics de fréquences cardiaques concomitante à des épisodes de dilatation pupilaires.

In [ ]:
import cv2
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
!pip install termcolor
from termcolor import colored
from datetime import datetime, date

### Manipulation du fichier fréquence cardiaque

In [ ]:
# fichier fréquence cardiaque

heart_rate = pd.read_csv("Frequence_cardiaque.csv")

print(heart_rate.head())
print(heart_rate.columns)


heart_rate['Timestamp']= pd.to_datetime(heart_rate['Timestamp'])
#print(heart_rate.head())

#calcul de la durée de l'enregistrement
duration_heart_rate = heart_rate['Timestamp'].iloc[-1] - heart_rate['Timestamp'].iloc[0]

#conversion ibi en bpm
heart_rate['bpm']= 60000 / heart_rate['ibi (ms)'] 

#fréquence cardiaque: temps relatif 
heart_rate['time_sec']= (heart_rate['Timestamp'] - heart_rate['Timestamp'].iloc[0]).dt.total_seconds() #relative time

print(f"Durée total de l'enregistrement: {duration_heart_rate.total_seconds()}")
print(f"Nombre total de battements: {len(heart_rate)} ")


### Manipulation du fichier vidéo eye tracking et extraction de données

In [ ]:
# Fichier vidéo Eye tracking 

video = cv2.VideoCapture("Mouvement_des_yeux.mp4")

#calcul de la dréquence d'images
fps= video.get(cv2.CAP_PROP_FPS) 

#calcul du nombre de frame
frame_count= int(video.get(cv2.CAP_PROP_FRAME_COUNT))

#calcul de la durée de la vidéo en secondes
duration_video = frame_count/fps

print(f"""FPS: {fps} 
Durée de l'enregistrement vidéo {duration_video} s""")

In [ ]:
#Sous échantillonage et extraction de données dans la video par traitement d'images 

frame_index = 0
sampling= 20 #fe=200/20= 10Hz selection d'1 frame toute les 20 frames sachant qu'on a environ 337 670frames dans la vidéo 
#sampling= 8 # Fe= 200/8= 25Hz
threshold= 50
min_area= 50
pupil_size = []
frame_times=[]

while True:
    ret, frame = video.read()
    if not ret:
        break
    #Sous échantillonnage     
    if frame_index % sampling != 0:
        frame_index += 1
        continue
        
    # Conversion  en niveau de gris    
    if len(frame.shape) == 3:  
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        frame_gray = frame  

    # Conversion en uint8
    frame_gray = frame_gray.astype(np.uint8)
    
    #Traitement d'image: thresholding et détection des contours
    _, thresh = cv2.threshold(frame_gray, threshold, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    diameter = 0 #diamètre des pupilles
    if contours:
        largest = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(largest)
        if area > min_area: 
            diameter = 2 * np.sqrt(area / np.pi)
     
    #stockage des données extraites
    
    pupil_size.append(diameter) #diamètre des pupilles
    frame_times.append(frame_index / fps)
    frame_index += 1

video.release()

eye_tracking =  pd.DataFrame({"frame_times": frame_times , "pupil_size" : pupil_size})


    
print(len(frame_times))
print(len(pupil_size))



In [ ]:
eye_tracking.describe()

### Vérification visuelle de la synchronisation des données de fréquence cardiaque et d'eye tracking

In [ ]:
#Vérification visuelle de la synchronisation des données

plt.figure(figsize=(12,5))
plt.plot(heart_rate['time_sec'], heart_rate['bpm'], label='HR')
plt.plot(frame_times, pupil_size, label='Pupil')
plt.legend()
plt.show()

## Identification et suppression des valeurs aberrantes de la fréquence cardiaque

In [ ]:
# Vue d'ensemble de la distribution de la fréquence cardiaque

heart_rate.describe()

hr_raw_len= len(heart_rate)

print(hr_raw_len)

In [ ]:
#Représentation de la distribution

plt.boxplot(heart_rate['bpm'])
plt.figure()
plt.plot(heart_rate['time_sec'], heart_rate['bpm'], label='HR')
plt.xlabel("Time_sec (s)")
plt.ylabel("bpm")
plt.legend()
plt.show()

In [ ]:
"""Suppression des valeus aberrantes de fréquence cardiaque (Fc)

Approche physiologique privilégiée avec : 
Fc_min = 40 bpm --> fréquence d'un athlète adulte au repos
Fc_max = 190 bpm --> Fc_max= 220-âge. ici l'âge choisi est 30 ans 

"""

heart_rate = heart_rate_clean = heart_rate[(heart_rate['bpm'] >= 40) & (heart_rate['bpm'] <= 190)]

heart_rate.describe()



In [ ]:
#Représentation de la distribution

plt.boxplot(heart_rate['bpm'])
plt.figure()
plt.plot(heart_rate['time_sec'], heart_rate['bpm'], label='HR')
plt.xlabel("Time_sec (s)")
plt.ylabel("bpm")
plt.legend()
plt.show()

## Traitement des valeurs répétées de la fréquence cardiaque

In [ ]:
#suppression des valeurs répétées selon le timestamp

heart_rate = heart_rate.drop_duplicates(subset='Timestamp')

hr_clean_len = len(heart_rate)

print(f"""Taille initiale de la distribution de la fréquence cardiaque: {hr_raw_len}
Taille de la distribution de la fréquence cardiaque après traitement des valeurs aberrantes et doublons {hr_clean_len}
Nombre de valeurs supprimées {hr_raw_len - hr_clean_len}""")

heart_rate.describe()


## Extraction des pics de fréquences cardiaques

In [ ]:
""""Extraction des pics de fréquences cardiaques

Principe : on va sur intervalle de confiance de 95% (soit moy +/- 2*ecart type).
Toute valeur au dessus de cet intervalle est considéré comme un pic de fréquence. 

""" 

threshold_hr = np.mean(heart_rate['bpm']) + 2 * np.std(heart_rate['bpm'])

peaks, properties = find_peaks(heart_rate['bpm'], height= threshold_hr)

hr_peak_time= heart_rate['time_sec'][peaks]
hr_peak=heart_rate['bpm'][peaks]
hr_peak = pd.DataFrame({"time_sec": hr_peak_time , "bpm" : hr_peak})
hr_peak_len= len(hr_peak)

hr_peak.describe()


In [ ]:
plt.figure()
plt.scatter(hr_peak['time_sec'], hr_peak['bpm'], label='HR')
plt.xlabel("Time_sec (s)")
plt.ylabel("bpm")
plt.legend()
plt.title("Pics de fréquence cardiaque")
plt.show()

## Génération automatique d'un compte rendu

In [ ]:
report = f"""
=== {colored("COMPTE RENDU D'ANALYSE PHYSIOLOGIQUE", attrs=["bold"])} ===

# {colored("Compte-rendu d'analyse cardiaque", attrs=["bold"])}
**{colored("Date :", attrs =["bold"])}** {datetime.now().date()}
**{colored("Session :", attrs =["bold"])}** ibi_data_{datetime.now().date()}_session

---

## {colored("Résumé des données", attrs=["bold"])}
-**{colored("Durée totale de l'enregistrement :", attrs =["bold"])}** {duration_heart_rate.total_seconds()//60} minutes and {duration_heart_rate.total_seconds()%60} secondes
-**{colored("Nombre total de battements :", attrs=["bold"])}** {hr_clean_len}
-**{colored("Fréquence cardiaque moyenne :", attrs=["bold"])}** {np.mean(heart_rate["bpm"])} bpm
-**{colored("Variabilité cardiaque(SDNN) :", attrs=["bold"])}** {np.std(heart_rate["ibi (ms)"])} ms
-**{colored("RMSSD :", attrs=["bold"])}** {round(np.sqrt(np.mean((np.diff(heart_rate["ibi (ms)"]))**2)))} ms

---

## {colored("Anomalies détectées", attrs=["bold"])}

-**{colored("Nombre de pics anormaux :", attrs=["bold"])}** {len(hr_peak)}
-**{colored("Nombre de groupes d'anomalies :", attrs=["bold"])}**
-**{colored("Durée moyenne des anolaies :", attrs=["bold"])}** secondes

"""

print(report)
